---
# Introduction à l'apprentissage automatique
---

<center><img src="https://python.gel.ulaval.ca/media/sio-u009/mlprocess_3.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

# Exemple 1 : Composantes d'un algorithme

Dans cet exercice, l'objectif est de définir et comprendre le vocabulaire utilisé en apprentissage automatique et ce qu'il représente. Nous allons voir l'ensemble des données (_data set_), la fonction de perte (_loss function_), la différence entre le vrai rique et le risque empirique, la régularisation, la fonction objectif (_objective function_).


Import des librairies nécessaires

In [ ]:
%matplotlib inline
# %matplotlib notebook # Pour la maniupulation des images 3D

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

Choix de la fonction à apprendre pour l'exemple : ici un polynome de degré 5.

In [ ]:
listeParamPoly = [0.03, 0.2, -1, -10, 100]

In [ ]:
def generate_data(N):
    x = np.random.uniform(-10,10,N)
    y = np.polyval(listeParamPoly,x) + np.random.normal(0.0, 15.0, N)
    return x.reshape(-1, 1), y

Génération des données : Choisir le nombre de points à utiliser :

In [ ]:
dataPoints = 20

In [ ]:
X,y = generate_data(dataPoints)

## Visualisation des données 

In [ ]:
fig, ax = plt.subplots()
# Affichage des points
ax.plot(X, y, 'o')
# Affichage de la fonction échantillonée
ax.plot(np.linspace(-10,10,100), np.polyval(listeParamPoly,np.linspace(-10,10,100)), color='black', linewidth=3)
# Axes & titre
ax.set_title('Data set')
ax.set_ylabel('y')
ax.set_xlabel('x')
plt.show()

## Définition de la fonction de perte

Choisir une fonction de perte parmi les trois données dans la présentation ... ou une autre au choix.

$$\mathcal{L}(\hat{y},y) = |\hat{y}-y|^2$$

In [ ]:
L = lambda y1, y2: np.abs(y1-y2)**2

Vous remarquerez que vous pouvez simplement passer de la perte quadratique à la perte absolue en enlevant le `**2` à la fin de la fonction et en rééxécutant la cellule.

## Définition du modèle prédictif 

On prend une régression linéaire pour l'exemple, si vous souhaitez changer ca pour un autre modèle c'est ici : 
$$\hat{y} = \theta_0\cdot x + \theta_1$$

Si on souhaitait prendre un modèle quadratique, on utiliserait : 
$$\hat{y} = \theta_0\cdot x^2 + \theta_1\cdot x + \theta_2$$

N'hésitez pas à changer le code ici un fois rendu à la fin de cet exercice pour voir ce que vous pourriez apprendre de mieux ;-)

Seul souci à prévoir : vous ne pourrez plus faire la visualisation 3D plus bas, car on aurais besoin de faire une visualisation en 4 dimensions.

In [ ]:
def h(x, theta):
    y = theta[0]*x + theta[1]
    return y

## La fonction de risque empirique

Le risque empirique est calculé à partir de la fonction de perte et du modèle choisis sur les données réelles.

Ici, nous avons choisi de prendre la moyenne des erreurs $\mathcal{L}(\hat{y},y)$ (MAE, MSE, ...) mais d'autre formes existente avec le $\min$ ou le $\max$.

In [ ]:
def risque_empirique(L,y,X,theta):
    loss = 0
    for i in range(0,np.max(y.shape)):
        y_hat = h(X[i], theta)
        loss = loss + L(y[i], y_hat)
    return loss/float(np.max(y.shape))

## Définition d'une fonction de régularisation

Ici une régularisation $L_1$ est choisie. Vous pouvez en prendre une autre (la $L_2$ par exemple).

In [ ]:
def regularisation(theta):
    return np.sum(np.abs(theta)) #L1
#    return ??? #L2

## La fonction objective

Se définit comme la somme du risque empirique et de la régularisation

In [ ]:
def objective(L,y,X,r,theta):
    return risque_empirique(L,y,X,theta) + r*regularisation(theta)

## Calcul exhaustif de la fonction objective (généralement intractable)

On calcule la fonction objectif pour tous les paramètres possibles. Avec deux ou trois paramètres c'est encore possible ... mais ce n'est visualisable qu'à deux.

In [ ]:
M = 25
linspace_A = np.linspace(-200, 200, M)
linspace_b = np.linspace(-200, 200, M)
R = np.zeros((M, M))
r = 0.1
for i in range(0,M):
    for j in range(0,M):
        theta = np.array([linspace_A[i], linspace_b[j]])
        R[i,j] = np.log(objective(L,y,X,r,theta)) 

## Visualisation de la fonction objective

Avec deux paramètres, ca se visualise bien. Comme dit tantot, si vous en avez mis plus, ceci ne pourra plus marcher.

In [ ]:
fig = plt.figure()
ax = plt.axes(projection='3d')
mesh_A, mesh_b = np.meshgrid(linspace_A, linspace_b)
ax.plot_surface(mesh_A, mesh_b, R)
plt.show()

## Optimisation avec scipy

On ne rentrera pas dans la boite (de Pandore) de l'optimisation dans cette formation. Nous allons utiliser un des outils de `scipy`, la méthode [`nelder-mead`](https://en.wikipedia.org/wiki/Nelder%E2%80%93Mead_method).

In [ ]:
from scipy.optimize import minimize

In [ ]:
r = 0.1
theta0 = np.array([0.0, 0.0])
f = lambda theta: objective(L,y,X,r,theta)
es = minimize(f, theta0, method='nelder-mead', options={'xtol': 1e-8, 'disp': True})
theta_opt = es.x

In [ ]:
print('Les paramètres estimés sont :')
print('A = %2.4f, b = %2.4f' % (theta_opt[0], theta_opt[1]))

## Visualisation du modèle appris

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.set_title('Polynôme')
ax.set_ylabel('y')
ax.set_xlabel('x')

linspace_x = np.linspace(-10, 10, num=100)
A = theta_opt[0]
b = theta_opt[1]
h_x = h(linspace_x,theta_opt)
ax.plot(linspace_x, h_x, color='red', linewidth=3)

plt.show()

## Calcul du "vrai" risque

Encore une fois, c'est rarement possible de le faire. Il faut avoir accès à la distribution réelle des données sur laquelle on intérroge notre modèle jusqu'à avoir une bonne estimation du risque. 

In [ ]:
X_test, y_test = generate_data(1000000)
vrai_risque = risque_empirique(L,y_test,X_test,theta_opt)
print('Le vrai risque du modèle appris est : %5.2f' % vrai_risque[0])

# Exemple 2 : Augmentation de la capacité d'un modèle

## Définition des fonctions de caractéristiques

Ajoutez dans le vecteur $\varphi$ autant de fonctions caractéristiques que vous le souhaitez. Idéalement non-linéaires.

In [ ]:
phi_names = [
       "x: x**0", 
       #"x: x**1",
       #...d'autres ?
       #"x: np.abs(x)",
       #"x: x > 0.0",
       #...au choix ?
       #"x: np.cos(x)",
       #"x: np.sin(0.1*x)"
       #...ou encore ?
      ]

phi = [
       lambda x: x**0, #...en haut c'était juste les noms pour l'affichage ... il faut les coder maintenant !
      ]

On va utiliser ce vecteur pour faire la projection. Voici la fonction de projection:

In [ ]:
def feature_space_projection(X, phi):
    X_features = []
    for i in range(0, len(phi)):
        X_features.append(np.apply_along_axis(phi[i], 0, X))
    X_augmented = np.concatenate(X_features, axis=1)
    return X_augmented

## Calcul de la projection dans l'espace des caractéristiques

Calcul de la projection puis affichage de notre entrée dans le nouvel espace.

In [ ]:
X_augmented = feature_space_projection(X, phi)

In [ ]:
X_augmented[0,:]

## Entraînement du modèle

Maintenant qu'on a ouvert la boîte à l'exercice précédent, on va utiliser le modèle de regression linéaire [`linear_model.LinearRegression()`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html#sklearn.linear_model.LinearRegression) de `scikit-learn`. On pourrait utiliser d'autres modèles [linéaires](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model).

In [ ]:
from sklearn import linear_model

reg = linear_model.LinearRegression()
reg.fit(X_augmented, y)

## Calcul de l'erreur sur l'ensemble d'entraînement vs le "vrai" risque

In [ ]:
from sklearn.metrics import mean_squared_error

### Erreur d'entraînement

Calcul de la fonction de perte [MSE](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html) sur les données d'entrainement:

In [ ]:
y_pred = reg.predict(X_augmented)
training_error = mean_squared_error(y, y_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

### Estimation du vrai risque

Et si on valide (non réalistiquement) sur la *vraie* distribution des données:

In [ ]:
X_test,y_test = generate_data(10000000)
X_test_augmented = feature_space_projection(X_test, phi)
y_test_pred = reg.predict(X_test_augmented)
test_error = mean_squared_error(y_test, y_test_pred)
print("Vrai risque du modèle appris est : %5.2f" % test_error)

## Attention au sur-apprentissage

Lorsque le nombre de caractéristiques utilisé est très important, la capacité augmente énormément jusqu'à apprendre *par coeur* les données.

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.set_title('Polynôme')
ax.set_ylabel('y')
ax.set_xlabel('x')

linspace_x = np.linspace(-10, 10, num=100000)
linspace_x = np.expand_dims(linspace_x, axis=1)

linspace_X_augmented = feature_space_projection(linspace_x, phi)

y_pred = reg.predict(linspace_X_augmented)
ax.plot(linspace_x, y_pred, color='red', linewidth=3)

plt.show()

## Régularisation et hyperparamètres

Il nous faut alors soit réduire le nombres de caractéristiques (et donc la dimensionnalité de l'espace de projection), soit *régulariser* nos paramètres pour assurer une bonne *généralisation*.

On va utiliser ici [ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html#sklearn.linear_model.ElasticNet) de `scikit-learn` qui permet de combiner les normes $L_1$ et $L_2$ pour la régularisation.

Vous pouvez chosir ici le taux de régularisation et la proportion de régularisation $L_1$ vs $L_2$.

In [ ]:
taux_de_regularisation = 1.0
ratio_normes = 0.5

On définit ensuite le modèle et on l'entraine.

In [ ]:
reg2 = linear_model.ElasticNet(alpha=taux_de_regularisation, copy_X=True, fit_intercept=True, l1_ratio=ratio_normes,
      max_iter=10000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)
reg2.fit(X_augmented, y)

Calcul du risque empirique : 

In [ ]:
y_pred = reg2.predict(X_augmented)
training_error = mean_squared_error(y, y_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

Calcul du *vrai* risque:

In [ ]:
X_test,y_test = generate_data(10000)
X_test_augmented = feature_space_projection(X_test, phi)
y_test_pred = reg2.predict(X_test_augmented)
test_error = mean_squared_error(y_test, y_test_pred)
print("Vrai risque du modèle appris est : %5.2f" % test_error)

Affichage du modèle régularisé :

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.set_title('Polynôme')
ax.set_ylabel('y')
ax.set_xlabel('x')

linspace_x = np.linspace(-10, 10, num=10000)
linspace_x = np.expand_dims(linspace_x, axis=1)

linspace_X_augmented = feature_space_projection(linspace_x, phi)

y_pred = reg2.predict(linspace_X_augmented)
ax.plot(linspace_x, y_pred, color='red', linewidth=3)

plt.show()

Choix fait des paramètres de régularisation par ElasticNet:

**Note :** Si vous avez des erreurs ici, assurez vous avant toute chose que le nombre de noms définis dans le dictionnaire `phi_names` est le même que le nombre de fonctions définies dans le dictionnaire `phi`.

In [ ]:
import pandas as pd

print('\''+pd.DataFrame({'names':phi_names,'coefs':reg2.coef_}).to_string(index=False)[1:])

**Pouvez vous identifier quels sont les fonctions de caractéristiques inutiles ?**

Si vous ne comprenez pas comment ou pourquoi, n'hésitez pas à venir en discuter dans le forum !

# Exemple 3 : Méthodologie de validation et test

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

In [ ]:
def confidence_interval(y, y_pred):
    n = len(y)
    s = np.sqrt(np.var(mean_squared_error(np.expand_dims(y,1).transpose(), np.expand_dims(y_pred,1).transpose(),
                          multioutput='raw_values'), ddof=1))
    return 1.96*s/np.sqrt(n)

(re)Génération des données, on va en prendre plus pour avoir de bons intervalles de confiance. Choisir le nombre de points à utiliser :

In [ ]:
dataPoints = 200

In [ ]:
X,y = generate_data(dataPoints)

In [ ]:
fig, ax = plt.subplots()
ax.plot(X, y, 'o')
ax.plot(np.linspace(-10,10,100), np.polyval(listeParamPoly,np.linspace(-10,10,100)), color='black', linewidth=3)
ax.set_title('Data set')
ax.set_ylabel('y')
ax.set_xlabel('x')
plt.show()

## Définition des fonctions de caractéristiques

Reprenez ici vos fonctions de caractéristiques de l'exercice précédent.

In [ ]:
# ??? Choisissez celles qui vous semble les plus pertinentes :-)
phi = [
    lambda x: x**0, #...Celle ci est pas très utile il me semble ...
    #...
]

## Préparation des données test

Après projection dans l'espace des caractéristiques, nous découpons notre dataset en trois parties.

In [ ]:
X_phi = feature_space_projection(X, phi)


### splits : train(50%) - validation(25%) - test(25%)

Un ensemble d'entrainement (_train_), un ensemble de _validation_ qui va nous permettre d'évaluer nos modèles et de choisir notre préféré sur un ensemble de données jamais vu, et un ensemble de _test_ pour l'évaluation final en mode **réel**.

Remarquez que *théoriquement*, l'ensemble de validation et l'ensemble de test sont identiques (sous l'hypothèse [IID](https://en.wikipedia.org/wiki/Independent_and_identically_distributed_random_variables)) et peuvent être permutés sans problème pour la méthodologie (mais au début seulement, après il est interdit de toucher au _test_ avant la fin).

In [ ]:
X_, X_test, y_, y_test = train_test_split(X_phi, y, test_size=0.25)
X_train, X_validation, y_train, y_validation = train_test_split(X_, y_, test_size=0.33)

### Entrainement du modèle sur les données d'entraînement

Vous pouvez ici changez les hyper-paramètres pour en voir l'effet sur les différentes erreurs sur les différents sous-ensembles de données. Vous pouvez aussi essayer d'autres [modèles linéaires](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model) (mais juste des regresseurs hein !).

In [ ]:
reg2 = linear_model.ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=100000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)
reg2.fit(X_train, y_train)

### Évaluation du modèle sur l'ensemble d'entraînement (training loss, erreur d'entrainement)

In [ ]:
y_train_pred = reg2.predict(X_train)
training_error = mean_squared_error(y_train, y_train_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

### Évaluation du modèle sur l'ensemble de validation (validation loss, erreur de validation)

In [ ]:
y_val_pred = reg2.predict(X_validation)
validation_error = mean_squared_error(y_validation, y_val_pred)
print("L'erreur de validation du modèle appris est : %5.2f" % validation_error)

### Évaluation du modèle sur l'ensemble de test (test loss, erreur de test)

In [ ]:
y_test_pred = reg2.predict(X_test)
test_error = mean_squared_error(y_test, y_test_pred)
print("L'erreur de test du modèle appris est : %5.2f" % test_error)

### Estimation du vrai risque du modèle

In [ ]:
X_risk, y_risk = generate_data(1000000)
X_risk = feature_space_projection(X_risk, phi)
y_risk_pred = reg2.predict(X_risk)
true_risk = mean_squared_error(y_risk, y_risk_pred)
print("L'erreur de généralisation du modèle appris est : %5.2f ± %2.2f" % (true_risk, confidence_interval(y_risk, y_risk_pred)))

Si on se rappelle que l'ensemble de test et de validation sont statistiquement identiques, on concoit aisément que l'estimation du vrai risque est toujours très difficile et potentiellement éloignée de l'erreur de validation ou de celle de test. 

On a *optimisé* plusieurs modèles sur l'ensemble d'entrainement, qu'on en a *choisi* les hyperparamètres d'apprentissage optimaux sur l'ensemble de validation et qu'on a vérifié nos choix **une seule fois** sur l'ensemble de test. 

# Exemple 4 : Méthodologie de validation croisée et test

Dans l'exercice précédent, nous avions un seul ensemble de _validation_, i.e. nos hyperparamètres d'apprentissage étaient optimaux pour cet ensembelde de données la et pouvait mal généraliser. La validation croisée vient pallier à ce problème en optimisant les hyeprparamètres sur plusieurs ensembles de validation **différents** pour plusieurs ensembles d'entrainement **différents**.

Pour ce faire, nous découpons seulement un ensemble de test pour calculer notre erreur empirique.

### splits : train(75%) - test(25%)

Un ensemble d'entrainement sur lequel nous ferons de la validation croisée et un ensemble de test pour la vérification *finale*.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_phi, y, test_size=0.25)

## Cross-validation du modèle/hyperparamètres

Vous pouvez ici changez les hyper-paramètres pour en voir l'effet sur les différentes erreurs sur les différents sous-ensembles de données. Vous pouvez aussi essayer d'autres modèles linéaires.

In [ ]:
reg2 = linear_model.ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=100000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)

In [ ]:
cv_score = cross_val_score(reg2, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

In [ ]:
-cv_score

In [ ]:
np.mean(-cv_score)

## Recherche en grille des hyperparamètres

Plutot que de rechercher *au jugé* les hyperparmètres, il est plus intéressant, quand c'est possible, de *tous* les essayer.

In [ ]:
reg2 = linear_model.ElasticNet(alpha=0.001, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)

Nous définissons donc ici, tous les couples d'hyperparmètres à utiliser.

In [ ]:
hyperparameters = {'l1_ratio':[0.0, 0.25, 0.5, 0.75, 1.0], 'alpha':[0.01, 0.1, 1, 10, 100]}

In [ ]:
clf = GridSearchCV(reg2, hyperparameters, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
clf.best_params_

Curieusement, quand on voit les résultats des *meilleurs* hyperparmètres, l'envie de raffiner votre grille de recherche dans la bonne région se fait probalement sentir ...

## Entraînement et évaluation sur le test

Une fois les meilleurs hyperparamètres choisis, on les utilise pour aprendre sur l'ensemble des données d'entrainement et vérifier une dernière fois notre apprentissage sur l'ensemble de test.

In [ ]:
best_alpha = clf.best_params_['alpha']
best_l1ratio = clf.best_params_['l1_ratio']

In [ ]:
reg2 = linear_model.ElasticNet(alpha=best_alpha, copy_X=True, fit_intercept=True, l1_ratio=best_l1ratio,
      max_iter=100000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='random', tol=0.0001, warm_start=False)
reg2.fit(X_train, y_train)

In [ ]:
y_train_pred = reg2.predict(X_train)
training_error = mean_squared_error(y_train, y_train_pred)
print("L'erreur d'entraînement du modèle appris est : %5.2f" % training_error)

In [ ]:
y_test_pred = reg2.predict(X_test)
test_error = mean_squared_error(y_test, y_test_pred)
print("L'erreur de test du modèle appris est : %5.2f ± %2.2f" % (test_error, confidence_interval(y_test, y_test_pred)))

On voit ici que l'intervalle de confiance et autour de $\pm 30\%$, impliquant que le *vrai* risque est probablement assez éloigné du risque obtenu.

Affichage du modèle appris :

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(X, y, 'o')
ax.plot(np.linspace(-10,10,100), np.polyval(listeParamPoly,np.linspace(-10,10,100)), color='black', linewidth=3)
ax.set_title('Polynôme')
ax.set_ylabel('y')
ax.set_xlabel('x')

linspace_x = np.linspace(-10, 10, num=100000)
linspace_x = np.expand_dims(linspace_x, axis=1)

linspace_X_augmented = feature_space_projection(linspace_x, phi)

y_pred = reg2.predict(linspace_X_augmented)
#ax.plot(linspace_x, y_pred, color='red', linewidth=3)

plt.show()